In [1]:
import re
import json
import random
from pprint import pprint
from ast import literal_eval
from collections import Counter, defaultdict

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

In [2]:
random.seed(2021)
np.random.seed(2021)

In [3]:
# each line in this table correspond to single tagging of an entry.
# an entry may have more than one tagging (e.g. a tag in 2D matrix, another in 1D matrix,
# secondary tags, etc. )
exportdata = pd.read_csv("exportdata.csv")
# I pulled entries related to projects chosen by Patrice
entries = pd.read_csv("entries_of_projects_chosen_by_patrice.csv")
# The widgets of AF's, i.e. 2D matrices, 1D matrices, secondary tagging widgets, etc.
af_widgets = pd.read_csv("af_widgets_of_interest.csv")
# Projects chosen by Patrice
projects = pd.read_csv("projects_chosen_by_patrice.csv")
#
exportables = pd.read_csv("af_exportables.csv")
# AF details
afs = pd.read_csv("analysis_frameworks.csv")
# user ids, first names and last names
users = pd.read_csv("user_names.csv")
# The matchin of pillar/sub-pillar names from different AFs into unified pillar/sub-pillar names
matching_2d_mat = pd.read_csv("mat_2d_matching.csv")
matching_1d_mat = pd.read_csv("mat_1d_matching.csv")

/home/abdullah/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
raw_name_to_tag_dict = {
    "Severity":"Severity",
    "Severity (Needs assessment entries only)":"Severity",
    "SEVERITY":"Severity",
    
    "Reliability":"Reliability",
    "RELIABILITY":"Reliability",
    
    "Demographic Groups":"Demographic Groups",
    "DEMOGRAPHIC GROUPS":"Demographic Groups",
    
    "Information date":"Information Date",
    "Information Date":"Information Date",
    "Date":"Information Date",
    "DATE OF INFORMATION":"Information Date",
    "Date range":"Information Date",
    
    "Geo Location":"Geo Location",
    "LOCATION":"Geo Location",
    "Geo location":"Geo Location",
    "Country":"Geo Location",
    "GEOLOCATIONS":"Geo Location",
    
    "Affected groups":"Affected Groups",
    "AFFECTED GROUPS":"Affected Groups",
    
    "Specific Needs Groups":"Specific Needs Groups",
    "SPECIFIC NEEDS GROUPS":"Specific Needs Groups",
}


In [5]:
def clean_titles(x):
    if x in list(raw_name_to_tag_dict.keys()):
        return raw_name_to_tag_dict[x]
    return x


af_widgets['title'] = af_widgets['title'].apply(clean_titles)

In [6]:
# widget names of 2D matrices in the chosen AFs's
mat2d_titles = [
    s.upper() for s in [
        #"Pre-Crisis",
        #"Shock/Event",
        #"In-Crisis",
        "Sectors",
        "Sectoral Information",
        "Matrix 2D",
    ]
]
# Pandas will read json objects as strings,
# here I am converting them into Python dict objects
af_widgets["properties"] = af_widgets["properties"].apply(json.loads)
##
mat2d_properties_ids = af_widgets[(
    af_widgets["title"].str.upper()).isin(mat2d_titles)][[
        "properties", "analysis_framework_id"
    ]]
mat2d_properties = mat2d_properties_ids["properties"].tolist()
mat2d_ids = mat2d_properties_ids["analysis_framework_id"].tolist()
## 1D Matrices
# widget names of 1D matrices in the chosen AFs's
mat1d_titles = [
    s.upper() for s in [
        "Operational Environment",  # iMMAP, 2020 Okular, Okular Generic, Colombia-AF, Nigeria Situation Analysis (OA), IFRC Master Framework 2019
        "Cross Sector",  #Rohingya Framework, IFRC Analytical Framework 2018, 
        "Matrix 1D",  # Situation Analysis Generic Yemen, 
        "Cross Sectors",  # Situation Analysis Generic Libya
    ]
]
mat1d_titles = mat1d_titles + [
    sub_s.title() for sub_s in ('Operational Environment - Dimension',
                                'Operational Environment - Subdimension')
]
mat1d_properties_ids = af_widgets[(
    af_widgets["title"].str.upper()).isin(mat1d_titles)][[
        "properties", "analysis_framework_id"
    ]]
mat1d_properties = mat1d_properties_ids["properties"].tolist()
mat1d_ids = mat1d_properties_ids["analysis_framework_id"].tolist()

In [7]:
afids_pillars_subpillars = dict()
errors = []
for mat, af_id in zip(mat2d_properties, mat2d_ids):
    try:
        dims = mat["data"]['dimensions']
        afids_pillars_subpillars[af_id] = {}
        for dim in dims:
            pillar = dim["title"]
            sub_pillars = []
            for sub_pillar_dict in dim["subdimensions"]:
                sub_pillars.append(sub_pillar_dict["title"])
            afids_pillars_subpillars[af_id][pillar] = sub_pillars
    except KeyError:
        errors.append([mat, af_id])
##
afids_rows_cells = dict()
for mat, af_id in zip(mat1d_properties, mat1d_ids):
    try:
        rows = mat["data"]['rows']
        afids_rows_cells[af_id] = {}
        for row in rows:
            pillar = row["title"]
            sub_pillars = []
            for sub_pillar_dict in row["cells"]:
                sub_pillars.append(sub_pillar_dict["value"])
            afids_rows_cells[af_id][pillar] = sub_pillars
    except KeyError:
        errors.append([mat, af_id])

In [8]:
def extract_title(x):
    if x["excel"].get("title"):
        return x["excel"]["title"]
    elif x["excel"].get("type") == "multiple":
        return x["excel"]["titles"]
##
exportables["data"] = exportables["data"].apply(literal_eval)
af_titles = exportables["data"].apply(extract_title).tolist()

In [9]:
print(entries.shape, exportdata.shape)
entries = entries[entries["entry_type"].eq("excerpt") & (~entries["excerpt"].isna())]
print(entries.shape, exportdata.shape)

(200243, 21) (1134710, 4)
(155743, 21) (1134710, 4)


In [10]:
exid_to_exdata = dict()
for ex_id, ex_data in zip(exportables["id"], exportables["data"]):
    exid_to_exdata[ex_id] = ex_data

In [11]:
widget_key_id_to_title = dict()
for w_key, w_id, title in zip(af_widgets["key"], af_widgets["widget_id"], af_widgets["title"]):
    widget_key_id_to_title[(w_key, w_id)] = title

In [12]:
def exportdata_to_tag_title(row):
    data = json.loads(row[1])
    wkey, wid = None, None
    if data.get("common"):
        wkey = data.get("common").get("widget_key")
        wid = data["common"].get("widget_id")
    if wkey and wid:
        if widget_key_id_to_title.get((wkey, wid)):
            return widget_key_id_to_title.get((wkey, wid))
    if data.get("report") and data["report"].get("other"):
            if len(data["report"]["other"]) == 1 and data["report"]["other"][0].get("title"):
                if data["report"]["other"][0]["title"]:
                    return data["report"]["other"][0]["title"]
    if isinstance(data.get("excel"), list) and len(data["excel"])==1 and \
     data["excel"][0].get("widget_key") and data["excel"][0].get("widget_id"):
        wkey = data["excel"][0]["widget_key"]
        wid = data["excel"][0]["widget_id"]
        if widget_key_id_to_title.get((wkey, wid)):
            return widget_key_id_to_title.get((wkey, wid))
    exportable_id = row[3]
    if exid_to_exdata[exportable_id]["excel"].get("title"):
        return exid_to_exdata[exportable_id]["excel"]["title"]
    elif exid_to_exdata[exportable_id]["excel"].get("type") == "multiple":
        return exid_to_exdata[exportable_id]["excel"]["titles"]
    raise

In [13]:
exportdata["tag_title"] = exportdata.apply(exportdata_to_tag_title, axis=1)

In [14]:
def exportdata_to_tag_value(row):
    data = json.loads(row[1])
    if isinstance(data["excel"], list) and len(data["excel"])==1:
        return data["excel"][0]["value"]
    if data["excel"].get("type") == "lists":
        return data["excel"]["values"]
    if isinstance(data["excel"], dict) and "value" in data["excel"]:
        return data["excel"].get("value")
    if isinstance(data["excel"], dict) and "values" in data["excel"]:
        return data["excel"].get("values")
    elif "values" in data["common"]:
        return data["common"]["values"]
    elif "value" in data["common"]:
        return data["common"]["value"]
    raise

In [15]:
exportdata["tag_value"] = exportdata.apply(exportdata_to_tag_value, axis=1)

In [16]:
def title_case(tag):
    if isinstance(tag, (list, tuple)):
        return tuple([x.title() for x in tag])
    return tag.title()

In [17]:
exportdata["tag_title"] = exportdata["tag_title"].apply(title_case)

In [18]:
mat_1d_and_flag_titles = [
    'Flag', 'Operational Environment', 'Cross Sector','Temática',
    ('Operational Environment - Dimension',
     'Operational Environment - Subdimension'),
    ('Flag - Dimension', 'Flag - Subdimension'),
    ('High Level Tags - Dimension', 'High Level Tags - Subdimension')
]

In [19]:
#exportdata.tag_title.unique()

In [20]:
#exportdata[exportdata["tag_title"].isin([('Operational Environment - Dimension', 'Operational Environment - Subdimension')])]["tag_value"].tolist()
#exportdata[exportdata["tag_title"].eq('High Level Tags')]["tag_value"].tolist()

In [21]:
mat2d_titles = [s.title() for s in mat2d_titles]
mat1d_titles = [
    s.title() if isinstance(s, str) else (sub_s.title() for sub_s in s)
    for s in mat1d_titles
]
mat_1d_and_flag_titles = [
    s.title() if isinstance(s, str) else (sub_s.title() for sub_s in s)
    for s in mat_1d_and_flag_titles
]

In [22]:
exportdata_of_interest = exportdata[exportdata["tag_title"].isin(
    mat2d_titles + mat_1d_and_flag_titles)]

In [23]:
exportdata_of_interest.shape, entries.shape

((208540, 6), (155743, 21))

In [24]:
entries_labeled = pd.merge(entries,
                           exportdata_of_interest,
                           how="inner",
                           left_on="id",
                           right_on="entry_id",
                           suffixes=('_entry', '_exportdata'))

In [25]:
entries_labeled["id_entry"].shape, entries_labeled["id_entry"].unique().shape

((157317,), (124247,))

In [26]:
def agg_group(group):
    tag_titles = group["tag_title"]
    tag_values = group["tag_value"]
    #
    grouped = defaultdict(list)
    for title, value in zip(tag_titles, tag_values):
        if title in mat_1d_and_flag_titles:
            grouped["Mat1D"].extend(value)
        elif title in mat2d_titles:
            grouped["Mat2D"].extend(value)
    return grouped

In [27]:
entries_labeled.columns

Index(['id_entry', 'created_at', 'modified_at', 'excerpt', 'image_raw',
       'analysis_framework_id', 'created_by_id', 'lead_id', 'modified_by_id',
       'entry_type', 'information_date', 'order', 'client_id', 'project_id',
       'tabular_field_id', 'dropped_excerpt', 'highlight_hidden', 'verified',
       'verification_last_changed_by_id', 'image_id', 'title', 'id_exportdata',
       'data', 'entry_id', 'exportable_id', 'tag_title', 'tag_value'],
      dtype='object')

In [28]:
tags = entries_labeled.groupby("id_entry").apply(agg_group)

In [29]:
entryids_tags = pd.DataFrame({"entry_id": tags.index, "tags": tags.values})

In [30]:
entryids_tags.shape

(124247, 2)

In [31]:
entries

,id,created_at,modified_at,excerpt,image_raw,analysis_framework_id,created_by_id,lead_id,modified_by_id,entry_type,...,order,client_id,project_id,tabular_field_id,dropped_excerpt,highlight_hidden,verified,verification_last_changed_by_id,image_id,title
0,25639,2019-03-24 11:25:28.055219+00,2019-03-25 04:27:17.715687+00,"On March 19, search-andrescue personnel airlif...",NaN,273,26,10607,26,excerpt,...,13,odi4z54i,878,NaN,NaN,False,False,NaN,NaN,"IFRC - Cyclone Idai, March 2019"
2,102051,2020-05-28 10:15:38.075272+00,2020-06-17 10:23:13.365974+00,"Currently, there are 139,771 confirmed cases i...",NaN,699,26,28940,657,excerpt,...,2,c5jrys5cq7vvjxju,1142,NaN,NaN,False,False,NaN,NaN,IFRC Turkey
4,21660,2019-02-08 11:27:52.756824+00,2019-02-08 11:27:53.091176+00,"According to The Guardian, more than 9,000 ref...",NaN,552,657,8644,657,excerpt,...,2,x8t1upes,788,NaN,"According to The Guardian, more than 9,000 ref...",False,False,NaN,NaN,Nigeria Situation Analysis (OA)
7,11245,2018-08-22 14:11:56.759863+00,2018-12-23 07:44:17.969986+00,Consideramos que tenemos que agotar nuestros r...,NaN,273,529,5274,529,excerpt,...,2,NaN,515,NaN,Consideramos que tenemos que agotar nuestros r...,False,False,NaN,NaN,Americas Regional Population Movement
9,96070,2020-04-16 03:56:55.509358+00,2020-04-16 05:46:07.41664+00,SANCAKTEPE SUB-PROVINCE: According to Baseline...,NaN,699,26,25511,26,excerpt,...,233,4ljiase9yfez69r9,1142,NaN,NaN,False,False,NaN,NaN,IFRC Turkey
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200238,412992,2021-08-26 11:17:51.374539+00,2021-08-26 11:17:51.619955+00,Une structure sanitiare est fonctionnelle à Dj...,NaN,1306,3331,63087,3331,excerpt,...,8,cm3d79d0kddyqzu2,2225,NaN,NaN,False,False,NaN,NaN,IMMAP/DFS RDC
200239,412991,2021-08-26 11:17:51.280929+00,2021-08-26 11:18:13.743401+00,"Dans la localité de Djaiba, les enseignants on...",NaN,1306,3331,63087,3331,excerpt,...,7,hyhsd18jbc2eokvo,2225,NaN,NaN,False,False,NaN,NaN,IMMAP/DFS RDC
200240,413010,2021-08-26 11:28:10.616471+00,2021-08-26 11:28:10.768735+00,L'activité économique a fortement ralenti en 2...,NaN,1306,3331,63089,3331,excerpt,...,4,n8dlm3201qf49png,2225,NaN,NaN,False,False,NaN,NaN,IMMAP/DFS RDC
200241,413011,2021-08-26 11:28:11.680335+00,2021-08-26 11:28:11.819706+00,"L'inflation a grimpé en flèche, alimentée par ...",NaN,1306,3331,63089,3331,excerpt,...,5,baoh85cumwf9clca,2225,NaN,NaN,False,False,NaN,NaN,IMMAP/DFS RDC


In [32]:
entries_ = entries.rename({"id": "entry_id"}, axis=1, inplace=False)
#
print(entries.shape)
#
entries_ = entries_[[
    'entry_id',
    'created_at',
    'modified_at',
    'excerpt',
    'entry_type',
    'analysis_framework_id',
    'created_by_id',
    'lead_id',
    'modified_by_id',
    'information_date',
    'order',
    'project_id',
    'verified',
]].drop_duplicates()
#
print(entries_.shape)

(155743, 21)
(155743, 13)


In [33]:
# print(entries_labeled.shape)
# entries_labeled = entries_labeled[[
#     'entry_id',
#     'created_at',
#     'modified_at',
#     'excerpt',
#     'entry_type',
#     'analysis_framework_id',
#     'created_by_id',
#     'lead_id',
#     'modified_by_id',
#     'information_date',
#     'order',
#     'project_id',
#     #'dropped_excerpt',
#     'verified',
# ]].drop_duplicates()
# print(entries_labeled.shape)

In [34]:
entries_labeled = pd.merge(entries_, entryids_tags, on="entry_id", how="inner")
entries_labeled.shape, entries_labeled.columns

((124247, 14),
 Index(['entry_id', 'created_at', 'modified_at', 'excerpt', 'entry_type',
        'analysis_framework_id', 'created_by_id', 'lead_id', 'modified_by_id',
        'information_date', 'order', 'project_id', 'verified', 'tags'],
       dtype='object'))

In [35]:
entries_labeled["tags_str"] = entries_labeled["tags"].apply(str)
entries_labeled.duplicated(subset=["entry_id", "tags_str"]).sum()

0

In [36]:
entries_labeled["entry_id"].duplicated().sum()

0

In [37]:
entries_labeled.shape

(124247, 15)

In [38]:
entries_labeled["tags"].isna().sum()

0

In [39]:
entries.shape, entries[~entries["excerpt"].isna()]["excerpt"].unique().shape

((155743, 21), (153382,))

In [40]:
entries_labeled.shape, entries_labeled.columns

((124247, 15),
 Index(['entry_id', 'created_at', 'modified_at', 'excerpt', 'entry_type',
        'analysis_framework_id', 'created_by_id', 'lead_id', 'modified_by_id',
        'information_date', 'order', 'project_id', 'verified', 'tags',
        'tags_str'],
       dtype='object'))

In [41]:
def tags_to_pillars(x):
    tags = x[0]
    af_id = x[1]
    pillars = []
    for tag_key, tag_value in tags.items():
        if tag_key != "Mat2D":
            continue
        if len(tag_value):
            for t in tag_value:
                if t[0]:
                    pillars.append(
                        str(af_id) + "->" + t[0].title() + "->" + t[1].title())
    return list(set(pillars))


def tags_to_sectors(x):
    tags = x[0]
    af_id = x[1]
    sectors = []
    for tag_key, tag_value in tags.items():
        if tag_key != "Mat2D":
            continue
        if len(tag_value):
            for t in tag_value:
                if len(t) > 2 and t[2]:
                    sectors.append(t[2].title())
    return list(set(sectors))


def tags_to_subsectors(x):
    tags = x[0]
    af_id = x[1]
    subsectors = []
    for tag_key, tag_value in tags.items():
        if tag_key != "Mat2D":
            continue
        if len(tag_value):
            subsectors = []
            for t in tag_value:
                if len(t) > 3 and t[3]:
                    if isinstance(t[3], list):
                        subsectors.extend([x.title() for x in t[3]])
                    else:
                        subsectors.append(t[3].title())
    return list(set(subsectors))


##
def tags_to_label2d(x):
    tags = x[0]
    af_id = x[1]
    label2d = []
    for tag_key, tag_value in tags.items():
        if tag_key != "Mat2D":
            continue
        if len(tag_value):
            for t in tag_value:
                if t[0]:
                    label2d.append("AF-ID:" + str(af_id) + "->Sector:" +
                                   t[2].title() + "->Pillar:" + t[0].title() +
                                   "->Sub-pillar:" + t[1].title())
    return list(set(label2d))


##
def tags_to_pillars_1d(x):
    tags = x[0]
    af_id = x[1]
    pillars_1d = []
    for tag_key, tag_value in tags.items():
        if tag_key != "Mat1D":
            continue
        if len(tag_value):
            for t in tag_value:
                if t[0]:
                    pillars_1d.append(
                        str(af_id) + "->" + t[0].title() + "->" + t[1].title())
    return list(set(pillars_1d))

In [42]:
entries_labeled.columns

Index(['entry_id', 'created_at', 'modified_at', 'excerpt', 'entry_type',
       'analysis_framework_id', 'created_by_id', 'lead_id', 'modified_by_id',
       'information_date', 'order', 'project_id', 'verified', 'tags',
       'tags_str'],
      dtype='object')

In [43]:
x = entries_labeled["tags"].tolist()[:3]

In [44]:
for k, v in x[1].items():
    print(k, v)

Mat2D [['Humanitarian Conditions', 'Physical & mental wellbeing', 'Health', []]]


In [45]:
entries_labeled["pillars"] = entries_labeled[["tags", 'analysis_framework_id'
                                              ]].apply(tags_to_pillars, axis=1)
entries_labeled["sectors"] = entries_labeled[["tags", 'analysis_framework_id'
                                              ]].apply(tags_to_sectors, axis=1)
entries_labeled["subsectors"] = entries_labeled[[
    "tags", 'analysis_framework_id'
]].apply(tags_to_subsectors, axis=1)
##
entries_labeled["label_2d"] = entries_labeled[[
    "tags", 'analysis_framework_id'
]].apply(tags_to_label2d, axis=1)
##
entries_labeled["pillars_1d"] = entries_labeled[["tags", 'analysis_framework_id'
                                              ]].apply(tags_to_pillars_1d, axis=1)

In [46]:
entries_labeled[~entries_labeled["excerpt"].isna()]["excerpt"].unique().shape

(122355,)

In [47]:
pills_unique_1d = set()
pills_occurances_1d = list()
for pills_1d in entries_labeled["pillars_1d"]:
    pills_unique_1d.update(pills_1d)
    pills_occurances_1d.extend(pills_1d)
len(pills_unique_1d), Counter(pills_occurances_1d).most_common()

(240,
 [('1306->Covid-19 Overview->Cases', 5353),
  ('1306->Displacement->Type/Numbers/Movements', 3639),
  ('1306->Context->Security & Stability', 3606),
  ('1306->Covid-19 Overview->Deaths', 3007),
  ('1306->Covid-19 Overview->Vaccination', 2663),
  ('829->Displacement ->Displacement', 2449),
  ('699->Humanitarian Profile->Affected Groups', 2439),
  ('1306->Covid-19 Containment Measures->Public Health Measures', 2215),
  ('1306->Context->Economy', 2161),
  ('1306->Casualties->Dead', 2107),
  ('829->Context->Legal Or Normative Framework', 1621),
  ('1306->Displacement->Push Factors', 1591),
  ('1306->Covid-19 Overview->Testing', 1580),
  ('699->Context->Demography', 1299),
  ('1306->Covid-19 Containment Measures->Movement Restrictions', 1247),
  ('537->Humanitarian Profile->Affected Groups', 1114),
  ('699->Humanitarian Profile->Casualties', 991),
  ('552->Humanitarian Profile->Affected Groups', 987),
  ('1306->Covid-19 Containment Measures->Physical Distancing', 857),
  ('1004->Displ

In [48]:
pills_unique = set()
pills_occurances = list()
for pills in entries_labeled["pillars"]:
    pills_unique.update(pills)
    pills_occurances.extend(pills)
Counter(pills_occurances).most_common()

[('1306->Humanitarian Conditions->Living Standards', 13535),
 ('1306->Impact->Drivers/Aggravating Factors', 9042),
 ('1306->Impact->Impact On System & Services', 8636),
 ('1306->Humanitarian Conditions->Physical & Mental Wellbeing', 8573),
 ('1306->Impact->Impact On People', 7887),
 ('829->Humanitarian Conditions->Living Standards', 6898),
 ('1306->At Risk->People At Risk / Vulnerable', 4976),
 ('1306->Capacities & Response->International', 4173),
 ('829->Response And Capacities->National And Local', 3634),
 ('1465->Humanitarian Situation->Humanitarian Profile', 2914),
 ('699->Capacities & Response->International Actors', 2602),
 ('699->Humanitarian Conditions->Living Standards', 2401),
 ('829->Humanitarian Conditions->Physical And Mental Well-Being', 2255),
 ('699->Capacities & Response->Government And Local Authorities', 2148),
 ('1306->Humanitarian Conditions->Coping Mechanisms', 2142),
 ('1306->Priorities->Priority Interventions (Staff)', 2134),
 ('829->Response And Capacities->Int

In [49]:
secs_unique = set()
secs_occurances = list()
for secs in entries_labeled["sectors"]:
    secs_unique.update(secs)
    secs_occurances.extend(secs)
#secs_unique
Counter(secs_occurances).most_common()

[('Health', 27271),
 ('Protection', 22014),
 ('Livelihoods', 12895),
 ('Cross', 11769),
 ('Food Security', 8460),
 ('Wash', 8251),
 ('Education', 8160),
 ('Shelter', 7713),
 ('Cross-Sector', 5906),
 ('Nutrition', 3457),
 ('Agriculture', 2953),
 ('Food', 2500),
 ('Logistics', 2417),
 ('Cross Cutting', 678),
 ('Logistic', 248),
 ('Livelihood', 132),
 ('Cccm', 125),
 ('Nfi', 113)]

In [50]:
entries_labeled.columns

Index(['entry_id', 'created_at', 'modified_at', 'excerpt', 'entry_type',
       'analysis_framework_id', 'created_by_id', 'lead_id', 'modified_by_id',
       'information_date', 'order', 'project_id', 'verified', 'tags',
       'tags_str', 'pillars', 'sectors', 'subsectors', 'label_2d',
       'pillars_1d'],
      dtype='object')

In [51]:
final_df = entries_labeled[entries_labeled["entry_type"].eq("excerpt")
                           & (~entries_labeled["excerpt"].isna())]
final_df.shape

(124247, 20)

In [52]:
def remove_newlines(excerpt):
    if not isinstance(excerpt, str):
        return excerpt
    return re.sub("\s+", " ", excerpt)

final_df.loc[:, 'excerpt'] = final_df['excerpt'].apply(remove_newlines)
#final_df.loc[:, 'dropped_excerpt'] = final_df['dropped_excerpt'].apply(remove_newlines)
##
final_df = final_df.sort_values("verified").reset_index(drop=True).drop_duplicates(subset='excerpt', keep="last")
final_df.shape, final_df.duplicated(subset="excerpt").sum()

((122247, 20), 0)

In [53]:
sector_name_mapper = {
    "Agriculture": "Agriculture",
    "Cccm": "",
    "Cross": "Cross",
    "Cross Cutting": "Cross",
    "Cross-Sector": "Cross",
    "Education": "Education",
    "Food": "Food Security",
    "Food Security": "Food Security",
    "Nutrition": "Nutrition",
    "Health": "Health",
    "Livelihood": "Livelihoods",
    "Livelihoods": "Livelihoods",
    "Logistic": "Logistics",
    "Logistics": "Logistics",
    "Protection": "Protection",
    "Shelter": "Shelter",
    "Shelter And Nfis": "Shelter",
    "Nfi": "",
    "Wash": "WASH",
}

In [54]:
af_title_to_id = dict()
for afid, title in zip(afs["id"], afs["title"]):
    af_title_to_id["title"] = afid

In [55]:
af_id_title = afs[["id", "title"]]
af_id_title["title"] = af_id_title["title"].str.title()
##
matching_2d_mat["Framework Title"] = matching_2d_mat["Framework Title"].str.title()
##
af_id_title.columns = ["analysis_framework_id", "Framework Name"]
matching_2d_mat = pd.merge(matching_2d_mat,
                    af_id_title,
                    how="left",
                    left_on="Framework Title",
                    right_on="Framework Name")
matching_2d_mat.columns

<ipython-input-55-bb9ead69f816>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  af_id_title["title"] = af_id_title["title"].str.title()


Index(['Unnamed: 0', 'Framework Title', 'Type', 'Pillar', 'Sub-pillar',
       'Virtual Type', 'Virtual Pillar', 'Virtual Sub-pillar', 'First-pass',
       'Verified', 'Reversible', 'Cover', 'analysis_framework_id',
       'Framework Name'],
      dtype='object')

In [56]:
matching_2d_mat["Pillar"] = matching_2d_mat["Pillar"].apply(
    lambda x: x.strip().title())
matching_2d_mat["Sub-pillar"] = matching_2d_mat["Sub-pillar"].apply(
    lambda x: x.strip().title())
matching_2d_mat["Final Pillar Name"] = matching_2d_mat[
    "Virtual Pillar"].apply(lambda x: x.strip().title())
matching_2d_mat["Final Sub-pillar Name"] = matching_2d_mat[
    "Virtual Sub-pillar"].apply(lambda x: x.strip().title())

In [57]:
#matching_2d_mat = matching_2d_mat[~matching_2d_mat["analysis_framework_id"].isna()]

matching_2d_mat = matching_2d_mat.astype({
    'Framework Name': str,
    'Pillar': str,
    'Sub-pillar': str,
    'Final Pillar Name': str,
    'Final Sub-pillar Name': str,
    'analysis_framework_id': int
})

In [58]:
matching_2d_mat["analysis_framework_id"] = matching_2d_mat["analysis_framework_id"].apply(str)
##
matching_2d_mat["original_pillar"] = matching_2d_mat[
    "analysis_framework_id"] + "->" + matching_2d_mat[
        "Pillar"] + "->" + matching_2d_mat["Sub-pillar"]
matching_2d_mat["original_subpillar"] = matching_2d_mat[
    "analysis_framework_id"] + "->" + matching_2d_mat[
        "Pillar"] + "->" + matching_2d_mat["Sub-pillar"]
##
matching_2d_mat["target_pillar"] = matching_2d_mat["Final Pillar Name"]
matching_2d_mat["target_subpillar"] = matching_2d_mat[
    "Final Pillar Name"] + "->" + matching_2d_mat["Final Sub-pillar Name"]

In [59]:
pillar_name_mapper = dict()
subpillar_name_mapper = dict()
for pillar, fpillar, subpillar, fsubpillar, af_id in zip(
        matching_2d_mat["original_pillar"], matching_2d_mat["target_pillar"],
        matching_2d_mat["original_subpillar"],
        matching_2d_mat["target_subpillar"],
        matching_2d_mat["analysis_framework_id"]):
    pillar_name_mapper[pillar] = fpillar
    subpillar_name_mapper[subpillar] = fsubpillar

In [60]:
matching_1d_mat["Framework Title"] = matching_1d_mat["Framework Title"].str.title()
##
matching_1d_mat = pd.merge(matching_1d_mat,
                           af_id_title,
                           how="left",
                           left_on="Framework Title",
                           right_on="Framework Name")


In [61]:
#matching_1d_mat = matching_1d_mat[~matching_1d_mat["analysis_framework_id"].isna()]
matching_1d_mat["analysis_framework_id"] = matching_1d_mat["analysis_framework_id"].apply(int)

In [62]:
matching_1d_mat.columns

Index(['Unnamed: 0', 'Framework Title', 'Type', 'Pillar', 'Sub-pillar',
       'Virtual Type', 'Virtual Pillar', 'Virtual Sub-pillar', 'First-pass',
       'Verified', 'Reversible', 'Cover', 'analysis_framework_id',
       'Framework Name'],
      dtype='object')

In [63]:
matching_1d_mat[matching_1d_mat["analysis_framework_id"].isna()]

,Unnamed: 0,Framework Title,Type,Pillar,Sub-pillar,Virtual Type,Virtual Pillar,Virtual Sub-pillar,First-pass,Verified,Reversible,Cover,analysis_framework_id,Framework Name


In [64]:
pillar_name_mapper_1d = dict()
subpillar_name_mapper_1d = dict()
for pillar, fpillar, subpillar, fsubpillar, af_id in zip(
        matching_1d_mat["Pillar"], matching_1d_mat["Virtual Pillar"],
        matching_1d_mat["Sub-pillar"],
        matching_1d_mat["Virtual Sub-pillar"], matching_1d_mat["analysis_framework_id"]):
    pillar = str(af_id) + "->" + pillar + "->" + subpillar
    pillar_name_mapper_1d[pillar] = fpillar
    subpillar_name_mapper_1d[pillar] = fpillar + "->" + fsubpillar

In [65]:
def sector_mapper(sec):
    if sec == sec:
        sectors =  [sector_name_mapper.get(s, "") for s in sec]
        return [sec for sec in sectors if sec]
    return []
##
def pillar_mapper(dim):
    if dim == dim:
        dim = [pillar_name_mapper.get(d, "") for d in dim]
        return [d for d in dim if d]
    return []
##
def subpillar_mapper(subdim):
    if subdim == subdim:
        subdim = [subpillar_name_mapper.get(d, "") for d in subdim]
        return [d for d in subdim if d]
    return []
##
def pillar_mapper_1d(pill):
    if pill == pill:
        pill = [pillar_name_mapper_1d.get(p, "") for p in pill]
        return [p for p in pill if p]
    return []
##
def subpillar_mapper_1d(subpill):
    if subpill == subpill:
        subpill = [subpillar_name_mapper_1d.get(p, "") for p in subpill]
        return [p for p in subpill if p]
    return []

In [66]:
final_df["sectors"] = final_df["sectors"].apply(sector_mapper)
final_df["subpillars_2d"] = final_df["pillars"].apply(subpillar_mapper)
final_df["pillars_2d"] = final_df["pillars"].apply(pillar_mapper)
##
final_df["subpillars_1d"] = final_df["pillars_1d"].apply(subpillar_mapper_1d)
final_df["pillars_1d"] = final_df["pillars_1d"].apply(pillar_mapper_1d)

In [67]:
final_df.columns

Index(['entry_id', 'created_at', 'modified_at', 'excerpt', 'entry_type',
       'analysis_framework_id', 'created_by_id', 'lead_id', 'modified_by_id',
       'information_date', 'order', 'project_id', 'verified', 'tags',
       'tags_str', 'pillars', 'sectors', 'subsectors', 'label_2d',
       'pillars_1d', 'subpillars_2d', 'pillars_2d', 'subpillars_1d'],
      dtype='object')

In [68]:
secs_unique = set()
secs_occurances = list()
for secs in final_df["sectors"]:
    secs_unique.update(secs)
    secs_occurances.extend(secs)
#secs_unique
Counter(secs_occurances).most_common()

[('Health', 26767),
 ('Protection', 21542),
 ('Cross', 17914),
 ('Livelihoods', 12830),
 ('Food Security', 10793),
 ('WASH', 8086),
 ('Education', 8051),
 ('Shelter', 7537),
 ('Nutrition', 3381),
 ('Agriculture', 2909),
 ('Logistics', 2616)]

In [69]:
pills_unique = set()
pills_occurances = list()
for pills in final_df["pillars_2d"]:
    pills_unique.update(pills)
    pills_occurances.extend(pills)
#pills_unique
Counter(pills_occurances).most_common()

[('Humanitarian Conditions', 46880),
 ('Impact', 34825),
 ('Capacities & Response', 14401),
 ('At Risk', 9274),
 ('Context', 3411),
 ('Priority Needs', 2657),
 ('Priority Interventions', 2447)]

In [70]:
subpills_unique = set()
subpills_occurances = list()
for subpills in final_df["subpillars_2d"]:
    subpills_unique.update(subpills)
    subpills_occurances.extend(subpills)
Counter(subpills_occurances).most_common()

[('Humanitarian Conditions->Living Standards', 26041),
 ('Humanitarian Conditions->Physical And Mental Well Being', 15546),
 ('Impact->Impact On Systems, Services And Networks', 12355),
 ('Impact->Driver/Aggravating Factors', 11815),
 ('At Risk->Risk And Vulnerabilities', 9273),
 ('Impact->Impact On People', 8404),
 ('Capacities & Response->International Response', 7606),
 ('Capacities & Response->National Response', 4191),
 ('Humanitarian Conditions->Coping Mechanisms', 4149),
 ('Capacities & Response->Number Of People Reached/Response Gaps', 2346),
 ('Impact->Number Of People Affected', 2251),
 ('Priority Interventions->Expressed By Humanitarian Staff', 2172),
 ('Priority Needs->Expressed By Humanitarian Staff', 1493),
 ('Priority Needs->Expressed By Population', 1164),
 ('Humanitarian Conditions->Number Of People In Need', 1144),
 ('Context->Security & Stability', 1025),
 ('Context->Economy', 858),
 ('Context->Socio Cultural', 577),
 ('Context->Environment', 327),
 ('Priority Interv

In [71]:
pills_unique_1d = set()
pills_occurances_1d = list()
for pills_1d in final_df["pillars_1d"]:
    pills_unique_1d.update(pills_1d)
    pills_occurances_1d.extend(pills_1d)
#pills_unique_1d
Counter(pills_occurances_1d).most_common()

[('Context', 17672),
 ('Displacement', 5711),
 ('Casualties', 3886),
 ('Humanitarian Access', 2191),
 ('Information and Communication', 825),
 ('Shock/Event', 763)]

In [72]:
subpills_unique_1d = set()
subpills_occurances_1d = list()
for subpills_1d in final_df["subpillars_1d"]:
    subpills_unique_1d.update(subpills_1d)
    subpills_occurances_1d.extend(subpills_1d)
#subpills_unique_1d
Counter(subpills_occurances_1d).most_common()

[('Context->Security & Stability', 5711),
 ('Context->Demography', 3779),
 ('Context->Economy', 3778),
 ('Casualties->Dead', 2996),
 ('Displacement->Type/Numbers/Movements', 2735),
 ('Displacement->Push factors', 1907),
 ('Context->Politics', 1808),
 ('Humanitarian Access->Physical constraints', 1740),
 ('Context->Socio Cultural', 1054),
 ('Context->Environment', 859),
 ('Context->Legal & Policy', 683),
 ('Casualties->Injured', 635),
 ('Displacement->Local integration', 534),
 ('Shock/Event->Type and characteristics', 451),
 ('Humanitarian Access->Number of people facing humanitarian access constraints/Humanitarian access gaps',
  451),
 ('Information and Communication->Knowledge and info gaps (pop)', 358),
 ('Displacement->Pull factors', 328),
 ('Information and Communication->Communication means and preferences', 314),
 ('Shock/Event->Hazard & Threats', 312),
 ('Casualties->Missing', 255),
 ('Displacement->Intentions', 207),
 ('Information and Communication->Information challenges an

In [73]:
final_df["subpillars_1d"].apply(lambda x:x!=[]).sum(),\
final_df["subpillars_2d"].apply(lambda x:x!=[]).sum(),\
final_df["sectors"].apply(lambda x:x!=[]).sum()

(25511, 79327, 90394)

### Splitting

In [ ]:
# add columns for sectors
for sector in secs_unique:
    final_df[sector] = 0
# add columns for dimensions
for pillar in pills_unique:
    final_df[pillar] = 0
# add columns for subdimensions
for subpillar in subpills_unique:
    final_df[subpillar] = 0
## add columns for 1d pillars
for pillar_1d in pills_unique_1d:
    final_df[pillar_1d] = 0
## add columns for 1d subpillars
for subpillar_1d in subpills_unique_1d:
    final_df[subpillar_1d] = 0

In [ ]:
for row in tqdm(final_df.iterrows(), total=final_df.shape[0]):
    sectors_i = row[1]["sectors"]
    pillars_i = row[1]["pillars"]
    subpillars_i = row[1]["subpillars"]
    pillars_1d_i = row[1]["pillars_1d"]
    subpillars_1d_i = row[1]["subpillars_1d"]
    idx = row[0]
    for sector in sectors_i:
        final_df.loc[idx, sector] = 1
    for pillar in pillars_i:
        final_df.loc[idx, pillar] = 1
    for subpillar in subpillars_i:
        final_df.loc[idx, subpillar] = 1
    for pillar_1d in pillars_1d_i:
        final_df.loc[idx, pillar_1d] = 1
    for subpillar_1d in subpillars_1d_i:
        final_df.loc[idx, subpillar_1d] = 1

In [ ]:
classes = list(secs_unique) + list(pills_unique) + list(
    subpills_unique) + list(pills_unique_1d) + list(subpills_unique_1d)
len(classes)

In [ ]:
final_df[classes].sum().astype(int)

### Stratified Split

In [ ]:
from skmultilearn.model_selection import iterative_train_test_split
from skmultilearn.model_selection.measures import get_combination_wise_output_matrix

In [ ]:
def train_val_test_df_split(df):
    class_to_id = {clss: i for i, clss in enumerate(classes)}
    num_classes = len(classes)
    labels = np.zeros([len(df), num_classes])
    for i, (sectors_i, pillars_i, subpillars_i, pillars_1d_i,
            subpillars_1d_i) in enumerate(
                zip(df["sectors"], df["pillars_2d"], df["subpillars_2d"],
                    df["pillars_1d"], df["subpillars_1d"])):
        for sec in sectors_i:
            labels[i, class_to_id[sec]] = 1
        for pil in pillars_i:
            labels[i, class_to_id[pil]] = 1
        for subpil in subpillars_i:
            labels[i, class_to_id[subpil]] = 1
        for pil_1d in pillars_1d_i:
            labels[i, class_to_id[pil_1d]] = 1
        for subpil_1d in subpillars_1d_i:
            labels[i, class_to_id[subpil_1d]] = 1
    ##
    X_train, y_train, X_test, y_test = iterative_train_test_split(
        df["excerpt"].to_numpy().reshape(-1, 1), labels, test_size=0.1)
    X_train, y_train, X_val, y_val = iterative_train_test_split(X_train,
                                                                y_train,
                                                                test_size=0.1)
    return {
        "X_train": X_train,
        "y_train": y_train,
        "X_test": X_test,
        "y_test": y_test,
        "X_val": X_val,
        "y_val": y_val
    }

In [ ]:
df = final_df[[
    'entry_id',
    'lead_id',
    'project_id',
    'analysis_framework_id',
    'excerpt',
    'created_by_id',
    'modified_by_id',
    'verified',
    'sectors',
    'pillars',
    'subpillars',
    'pillars_1d',
    'subpillars_1d',
]]
df.columns = [
    'entry_id',
    'lead_id',
    'project_id',
    'analysis_framework_id',
    'excerpt',
    'created_by_id',
    'modified_by_id',
    'verified',
    'sectors',
    'pillars_2d',
    'subpillars_2d',
    'pillars_1d',
    'subpillars_1d',
]

In [ ]:
X_train, y_train, X_test, y_test, X_val, y_val = list(
    train_val_test_df_split(df).values())

In [ ]:
final_df_train = final_df[final_df["excerpt"].isin(X_train.reshape(-1))]
final_df_val = final_df[final_df["excerpt"].isin(X_val.reshape(-1))]
final_df_test = final_df[final_df["excerpt"].isin(X_test.reshape(-1))]

In [ ]:
df_train = df[df["excerpt"].isin(X_train.reshape(-1))]
df_val = df[df["excerpt"].isin(X_val.reshape(-1))]
df_test = df[df["excerpt"].isin(X_test.reshape(-1))]

In [ ]:
df_train.index & df_val.index, df_train.index & df_test.index, df_test.index & df_val.index 

In [ ]:
df_train.to_csv("data_v0.5.1_train.csv", index=None)
df_val.to_csv("data_v0.5.1_val.csv", index=None)
df_test.to_csv("data_v0.5.1_test.csv", index=None)

In [ ]:
df.to_csv("data_v0.5.1_full.csv", index=None)